<a href="https://colab.research.google.com/github/abhigyanpal1/wallet-risk-scoring/blob/main/Zeru_assignment_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Wallet Risk Scoring From Scratch

In this notebook we will:

1. **Load** a list of wallet addresses  
2. **Fetch** on‑chain transaction history (via Covalent or Etherscan APIs)  
3. **Compute** per‑wallet risk features  
4. **Normalize** these features and **aggregate** into a 0–1000 score  
5. **Export** `wallet_id, score` as a CSV  

---

## 1. Install & Import Dependencies


In [1]:
# Install any APIs / SDKs you need
!pip install pandas requests sklearn

# Imports
import os
import time
import requests
import pandas as pd
from sklearn.preprocessing import MinMaxScaler


  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


## 2. Set Your API Keys


In [5]:
# Covalent (or Etherscan) API key
ETHERSCAN_API_KEY = "RMYHTZKN3ZRVSIM6SSPVH7DKZVAXD9EIPU"
CHAIN_ID = 1   # 1 = Ethereum mainnet


## 3. Load Wallet IDs
You can either upload your `Wallet id.xlsx` via the Colab sidebar, or read from Drive.


In [4]:
from google.colab import files

uploaded = files.upload()


# Read into DataFrame
wallets_df = pd.read_excel("Wallet id.xlsx")
# Expect a column named "wallet_id"
wallets_df.head()


Saving Wallet id.xlsx to Wallet id.xlsx


,wallet_id
0,0x0039f22efb07a647557c7c5d17854cfd6d489ef3
1,0x06b51c6882b27cb05e712185531c1f74996dd988
2,0x0795732aacc448030ef374374eaae57d2965c16c
3,0x0aaa79f1a86bc8136cd0d1ca0d51964f4e3766f9
4,0x0fe383e5abc200055a7f391f94a5f5d1f844b9ae


## 4. Fetch Transaction History Function

Here we use Covalent’s `/transactions_v2/` endpoint.  
Feel free to swap in Etherscan or TheGraph if you prefer.


In [6]:
# 4️⃣ Fetch Txns via Etherscan
def fetch_txns_for_wallet(address, api_key=ETHERSCAN_API_KEY):
    """
    Uses Etherscan 'txlist' endpoint to pull up to 10k
    normal transactions for a single address.
    """
    url = (
      f"https://api.etherscan.io/api"
      f"?module=account"
      f"&action=txlist"
      f"&address={address}"
      f"&startblock=0"
      f"&endblock=99999999"
      f"&page=1"
      f"&offset=10000"
      f"&sort=asc"
      f"&apikey={api_key}"
    )
    resp = requests.get(url)
    data = resp.json()
    # Etherscan returns `result` list on success
    if data["status"] != "1":
        return pd.DataFrame()
    return pd.DataFrame(data["result"])

# Quick smoke‑test
sample = wallets_df["wallet_id"].iloc[0]
txns = fetch_txns_for_wallet(sample)
print(f"Got {len(txns)} txns for {sample}")
txns.head()


Got 1946 txns for 0x0039f22efb07a647557c7c5d17854cfd6d489ef3


,blockNumber,blockHash,timeStamp,hash,nonce,transactionIndex,from,to,value,gas,gasPrice,input,methodId,functionName,contractAddress,cumulativeGasUsed,txreceipt_status,gasUsed,confirmations,isError
0,3606695,0x3933a1503dfd71252ed989c8137cd888dc6f460a0eee...,1493254201,0x9da9da95c9c48d82a818d60e13c14981264bb8b8418b...,289758,37,0x32be343b94f860124dc4fee278fdcbd38c102d88,0x0039f22efb07a647557c7c5d17854cfd6d489ef3,200000000000000000,333333,30000000000,0x,0x,,,962090,,21000,19393970,0
1,3606709,0xb6de24eb2bfbcbcf439647755ae5382b09cef95769f7...,1493254443,0x48154e11e7844969545339b8328748bf05c49841674e...,0,71,0x0039f22efb07a647557c7c5d17854cfd6d489ef3,0x9ae98746eb8a0aeee5ff2b6b15875313a986f103,7000000000000000,27961,20000000000,0x338cdca1,0x338cdca1,request(),,2094834,,23301,19393956,0
2,3606713,0x2b96e45695f1db42ce79ad0433a282d35d14caaff35b...,1493254495,0xe17bf3ca4d4616c89c3cefba65077420eecac2e115fd...,1,22,0x0039f22efb07a647557c7c5d17854cfd6d489ef3,0x9ae98746eb8a0aeee5ff2b6b15875313a986f103,7000000000000000,27961,20000000000,0x338cdca1,0x338cdca1,request(),,2683717,,23301,19393952,0
3,3606717,0xf5b41d0a8d0617b1d4b2bdbbdcb5bde641b93d9e7081...,1493254558,0xaf10622c370d28f2e1f2631efbc02c35e296a7d6dbd1...,2,22,0x0039f22efb07a647557c7c5d17854cfd6d489ef3,0x9ae98746eb8a0aeee5ff2b6b15875313a986f103,0,61078,20000000000,0x797af6271f6c8b760228ef69a00bea01f72146a5e5ce...,0x797af627,confirm(bytes32 _h),,752334,,35898,19393948,0
4,3622170,0x2849e6679136438e43daf9484c7a1b15d5159cda01a6...,1493490509,0x42e2cf2103643a349594a0135d5842e02f643ba578e7...,3,115,0x0039f22efb07a647557c7c5d17854cfd6d489ef3,0x0b8d56c26d8cf16fe1bddf4967753503d974de06,20000000000000000,120763,20000000000,0x29cbdc860000000000000000000000000039f22efb07...,0x29cbdc86,"buyin(address _who, uint256 _maxPrice)",,2981051,,100636,19378495,0


## 5. Feature Engineering

As an example, we’ll compute:
- **total_txn_count**  
- **txn_rate** (txns per day since first txn)  
- **unique_counterparties**  
- **avg_value_eth**  
- **pct_failed_txns**  




In [7]:
# 5️⃣ Compute Risk Features (adjusted for Etherscan fields)
def compute_risk_features(tx_df):
    # if empty, return zeros
    if tx_df.empty:
        return dict(total_txn_count=0,
                    txn_rate=0,
                    unique_counterparties=0,
                    avg_value_eth=0,
                    pct_failed_txns=0)

    # convert timestamp
    tx_df["block_signed_at"] = pd.to_datetime(
        tx_df["timeStamp"].astype(int), unit="s"
    )

    total = len(tx_df)
    span_days = max(
        (tx_df["block_signed_at"].max() - tx_df["block_signed_at"].min()).days,
        1
    )

    # counterparties are the `to` field
    counterparties = tx_df["to"].fillna("").unique()

    # value is in Wei
    vals_eth = tx_df["value"].astype(float) / 1e18

    # failed if isError == "1"
    failed_pct = (tx_df["isError"] == "1").mean()

    return {
        "total_txn_count": total,
        "txn_rate": total / span_days,
        "unique_counterparties": len(counterparties),
        "avg_value_eth": vals_eth.mean(),
        "pct_failed_txns": failed_pct
    }

# test
print(compute_risk_features(txns))


{'total_txn_count': 1946, 'txn_rate': 0.7743732590529248, 'unique_counterparties': 396, 'avg_value_eth': np.float64(4.393224449195074), 'pct_failed_txns': np.float64(0.025693730729701953)}


## 6. Build Feature Matrix for All Wallets


In [8]:
all_features = []
for addr in wallets_df["wallet_id"]:
    df = fetch_txns_for_wallet(addr)
    feats = compute_risk_features(df)
    feats["wallet_id"] = addr
    all_features.append(feats)
    time.sleep(0.2)  # rate‑limit

features_df = pd.DataFrame(all_features)
features_df.head()


,total_txn_count,txn_rate,unique_counterparties,avg_value_eth,pct_failed_txns,wallet_id
0,1946,0.774373,396,4.393224,0.025694,0x0039f22efb07a647557c7c5d17854cfd6d489ef3
1,4,4.000000,4,0.006075,0.000000,0x06b51c6882b27cb05e712185531c1f74996dd988
2,3,3.000000,3,0.006200,0.000000,0x0795732aacc448030ef374374eaae57d2965c16c
3,24,0.012773,13,8.407646,0.000000,0x0aaa79f1a86bc8136cd0d1ca0d51964f4e3766f9
4,4,0.063492,4,0.006501,0.000000,0x0fe383e5abc200055a7f391f94a5f5d1f844b9ae


## 7. Normalize & Score

We scale each feature 0–1, then combine with weights and multiply to get 0–1000.


In [9]:
# Choose weights for each feature (sum to 1)
weights = {
    "total_txn_count": 0.2,
    "txn_rate": 0.2,
    "unique_counterparties": 0.2,
    "avg_value_eth": 0.2,
    "pct_failed_txns": 0.2
}

# Normalize
scaler = MinMaxScaler()
scaled = scaler.fit_transform(features_df[list(weights)])
scaled_df = pd.DataFrame(scaled, columns=list(weights), index=features_df.index)

# weighted sum
features_df["raw_score"] = sum(
    scaled_df[f] * w for f, w in weights.items()
)

# scale to 0–1000
features_df["score"] = (features_df["raw_score"] * 1000).round().astype(int)

features_df[["wallet_id", "score"]].head()


,wallet_id,score
0,0x0039f22efb07a647557c7c5d17854cfd6d489ef3,440
1,0x06b51c6882b27cb05e712185531c1f74996dd988,135
2,0x0795732aacc448030ef374374eaae57d2965c16c,101
3,0x0aaa79f1a86bc8136cd0d1ca0d51964f4e3766f9,22
4,0x0fe383e5abc200055a7f391f94a5f5d1f844b9ae,4


## 8. Export CSV


In [10]:
out = features_df[["wallet_id", "score"]]
out.to_csv("wallet_risk_scores.csv", index=False)
files.download("wallet_risk_scores.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [11]:
df = pd.read_csv("wallet_risk_scores.csv")

# Display the first few rows (or drop .head() to show all)
df.head(20)

,wallet_id,score
0,0x0039f22efb07a647557c7c5d17854cfd6d489ef3,440
1,0x06b51c6882b27cb05e712185531c1f74996dd988,135
2,0x0795732aacc448030ef374374eaae57d2965c16c,101
3,0x0aaa79f1a86bc8136cd0d1ca0d51964f4e3766f9,22
4,0x0fe383e5abc200055a7f391f94a5f5d1f844b9ae,4
5,0x104ae61d8d487ad689969a17807ddc338b445416,2
6,0x111c7208a7e2af345d36b6d4aace8740d61a3078,168
7,0x124853fecb522c57d9bd5c21231058696ca6d596,202
8,0x13b1c8b0e696aff8b4fee742119b549b605f3cbc,101
9,0x1656f1886c5ab634ac19568cd571bc72f385fdf7,54
